In [1]:
import pandas as pd
from dw import lt
import logging
import random
logging.basicConfig(
    level = logging.INFO
    ,format = "%(asctime)s [%(levelname)s] %(message)s")

In [2]:
n = 10000
df = pd.DataFrame(
    {
        "id": range(n)
        ,"score": [random.random() for i in range(n)]
        ,"amt": [random.choice(range(1000)) for i in range(n)]
        ,"cat": [random.choice(["test", "train"]) for i in range(n)]
        ,'time': [
                random.choice(
                    [pd.Timestamp(i) for i in 
                        ["20130102","20130202","20130302"]]
                ) 
                for i in range(n)]
    }
)
df.head()

,id,score,amt,cat,time
0,0,0.412243,972,train,2013-03-02
1,1,0.795561,536,train,2013-01-02
2,2,0.743286,457,test,2013-03-02
3,3,0.477455,445,train,2013-03-02
4,4,0.323246,893,train,2013-03-02


In [3]:
lt.drop('test')
lt.create(
        tbl_nme = 'test'
        ,dtypes = {
            'id':'integer'
            ,'score':'real'
            ,'amt':'integer'
            ,'cat':'text'
            ,'time':'text'
            ,'constraint df_pk':
                'primary key (id)'
        }
    )
lt.write(df,'test')

2022-01-23 01:08:13,174 [INFO] running:
drop table test
2022-01-23 01:08:13,179 [INFO] running:
create table test(
    id integer
    ,score real
    ,amt integer
    ,cat text
    ,time text
    ,constraint df_pk primary key (id)
)
2022-01-23 01:08:13,180 [INFO] done
2022-01-23 01:08:13,293 [INFO] running:
insert into test (id,score,amt,cat,time) values (:id,:score,:amt,:cat,:time)
2022-01-23 01:08:13,294 [INFO] len(args) = 10000
2022-01-23 01:08:13,387 [INFO] done


In [4]:
lt.qry('test').where("score > 0.5") \
.valc('time,cat',"avg(score) avgscore, round(sum(amt)/1e3,2) total") \
.pivot('time','cat',['n','avgscore','total'])

2022-01-23 01:08:13,407 [INFO] running:
with x as (
    select * from test
    where score > 0.5
)
select 
    time,cat
    ,count(1) n
    ,avg(score) avgscore, round(sum(amt)/1e3,2) total
from x
group by time,cat
order by n desc
2022-01-23 01:08:13,413 [INFO] done


n         avgscore             total        
cat          test  train      test     train    test   train
time                                                        
2013-01-02  816.0  847.0  0.746747  0.750452  398.34  417.31
2013-02-02  837.0  858.0  0.748214  0.743094  419.11  447.04
2013-03-02  805.0  860.0  0.756775  0.739017  394.89  422.35

In [5]:
lt.run('select * from test limit 2')
lt.run('select * from test where score > :score limit 2'
    ,args = {'score':'0.9'})
lt.run('select * from test where score > :score limit 2'
    ,score = 0.9)

2022-01-23 01:08:13,437 [INFO] running:
select * from test limit 2
2022-01-23 01:08:13,439 [INFO] done
2022-01-23 01:08:13,442 [INFO] running:
select * from test where score > :score limit 2
2022-01-23 01:08:13,443 [INFO] len(args) = 1
2022-01-23 01:08:13,444 [INFO] done
2022-01-23 01:08:13,446 [INFO] running:
select * from test where score > 0.9 limit 2
2022-01-23 01:08:13,448 [INFO] done


,id,score,amt,cat,time
0,12,0.938176,499,train,2013-03-02
1,22,0.996893,641,train,2013-03-02


In [6]:
lt.qry('test').top()

2022-01-23 01:08:13,469 [INFO] running:
with x as (
    select * from test
)
select * from x limit 1
2022-01-23 01:08:13,471 [INFO] done


id                0
score      0.412243
amt             972
cat           train
time     2013-03-02
Name: 0, dtype: object

In [7]:
lt.qry('test').head()

2022-01-23 01:08:13,486 [INFO] running:
with x as (
    select * from test
)
select * from x limit 5
2022-01-23 01:08:13,487 [INFO] done


,id,score,amt,cat,time
0,0,0.412243,972,train,2013-03-02
1,1,0.795561,536,train,2013-01-02
2,2,0.743286,457,test,2013-03-02
3,3,0.477455,445,train,2013-03-02
4,4,0.323246,893,train,2013-03-02


In [8]:
lt.qry('test').len()

2022-01-23 01:08:13,501 [INFO] running:
with x as (
    select * from test
)
select count(1) from x
2022-01-23 01:08:13,503 [INFO] done


10000

In [9]:
lt.qry('test').select("id,score,amt").top()
lt.qry('test').select(["id","score","amt"]).top()
lt.qry('test').select("id","score","amt").top()

2022-01-23 01:08:13,518 [INFO] running:
with x as (
    select id,score,amt
    from test
)
select * from x limit 1
2022-01-23 01:08:13,520 [INFO] done
2022-01-23 01:08:13,522 [INFO] running:
with x as (
    select id,score,amt
    from test
)
select * from x limit 1
2022-01-23 01:08:13,523 [INFO] done
2022-01-23 01:08:13,526 [INFO] running:
with x as (
    select id,score,amt
    from test
)
select * from x limit 1
2022-01-23 01:08:13,527 [INFO] done


id         0.000000
score      0.412243
amt      972.000000
Name: 0, dtype: float64

In [10]:
(
    lt.qry('test x')
    .select('x.id','y.id as yid','x.score','z.score as zscore')
    .join("test y","x.id = y.id+1","x.id <= y.id+1")
    .join("test z","x.id = z.id+2","x.id >= z.id+1")
    .where('x.id < 10','z.id < 10')
    .head()
)

2022-01-23 01:08:13,548 [INFO] running:
with x as (
    select x.id,y.id as yid,x.score,z.score as zscore
    from test x
    left join test y
        on x.id = y.id+1
        and x.id <= y.id+1
    left join test z
        on x.id = z.id+2
        and x.id >= z.id+1
    where x.id < 10
        and z.id < 10
)
select * from x limit 5
2022-01-23 01:08:13,550 [INFO] done


,id,yid,score,zscore
0,2,1,0.743286,0.412243
1,3,2,0.477455,0.795561
2,4,3,0.323246,0.743286
3,5,4,0.803028,0.477455
4,6,5,0.364130,0.323246


In [11]:
(
    lt.qry('test x')
    .select('x.cat,y.cat as bcat'
        ,'sum(x.score) bscore','sum(y.score) yscore','count(1) n')
    .join("test y","x.id = y.id+1")
    .where('x.id < 1000')
    .group_by('x.cat,y.cat')
    .having('count(1) > 50','sum(y.score) > 100')
    .order_by('x.cat desc','sum(y.score) desc')
    .run()
)

2022-01-23 01:08:13,564 [INFO] running:
select x.cat,y.cat as bcat,sum(x.score) bscore,sum(y.score) yscore,count(1) n
from test x
left join test y
    on x.id = y.id+1
where x.id < 1000
group by x.cat,y.cat
having count(1) > 50
    and sum(y.score) > 100
order by x.cat desc,sum(y.score) desc
2022-01-23 01:08:13,974 [INFO] done


,cat,bcat,bscore,yscore,n
0,train,train,136.793593,133.806966,265
1,train,test,125.064553,125.036549,245
2,test,train,128.581954,128.463423,246
3,test,test,121.019161,124.122500,243
